# Comparing Optimizers in TensorFlow ANN
This notebook trains the same ANN model using different optimizers and compares their accuracy.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
dataset = pd.read_csv('/content/cell_data.csv')
dataset.replace('?', np.nan, inplace=True)
dataset.dropna(inplace=True)
dataset['BareNuc'] = dataset['BareNuc'].astype(int)
dataset.rename(columns={'Temp Class': 'Class'}, inplace=True)
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Define optimizers
optimizers = [
    ('SGD', tf.keras.optimizers.SGD()),
    ('RMSprop', tf.keras.optimizers.RMSprop()),
    ('Adagrad', tf.keras.optimizers.Adagrad()),
    ('Adadelta', tf.keras.optimizers.Adadelta()),
    ('Nadam', tf.keras.optimizers.Nadam()),
    ('FTRL', tf.keras.optimizers.Ftrl())
]

# Store results
results = []

for name, optimizer in optimizers:
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(6, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=0)
    y_pred = model.predict(X_test) > 0.5
    acc = accuracy_score(y_test, y_pred)
    results.append((name, acc * 100))

In [ ]:
# Display results as table
results_df = pd.DataFrame(results, columns=['Optimizer', 'Accuracy (%)'])
results_df.sort_values(by='Accuracy (%)', ascending=False).reset_index(drop=True)